In [1]:
from utilities.utils_clean import *
from utilities.test_dataset_utils import *

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\amoha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amoha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

#download_patents_pto(start_year=2015, end_year=2015, kind='grant',download_path="data")
#unzip_files("data","patent_grants_2015")

In [ ]:
# Example usage
test_dataset = create_test_dataset_from_freilich( #"patent_xmls_2010.json"
    year=2015,  # The year you want to analyze
    freilich_data_path="Freilich.Data.Compressed.xlsb",  # Path to your Freilich dataset
    path_to_all_xmls_for_chosen_year="../app/data/patent_grants_2015"  # Directory containing XML files
)

# # Optional: Check the results
# print(f"Number of patents extracted: {len(test_dataset)}")
# print("Sample document numbers:", list(test_dataset.keys())[:5])

In [3]:
save_as_json(test_dataset, "patent_xmls_2015.json")

Saved as patent_xmls_2015.json


In [2]:
data = read_json("patent_xmls_2015.json")
df = pd.DataFrame.from_dict(data,orient="index").reset_index()
df.columns = ["patentnumber","xml"]
df.head()

,patentnumber,xml
0,RE045323,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
1,RE045324,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
2,RE045325,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
3,8925349,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
4,8925551,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."


In [5]:

soup = BeautifulSoup(df[df["patentnumber"]=="8927546"].xml.values[0], "xml")
siblings = soup.find_all(["heading", "p"])
examples = extract_examples_start_w_word(siblings)

In [2]:
from collections import Counter
import nltk
from nltk import pos_tag, word_tokenize
def analyze_sentence_tense(text, threshold=0.5):
    text = text.replace("  ", "").replace("\n", " ").replace("\t", " ")

    # Ensure required NLTK data is available
    try:
        nltk.data.find("taggers/averaged_perceptron_tagger")
    except LookupError:
        nltk.download("averaged_perceptron_tagger")
        nltk.download("punkt")

    # Tokenize and POS tag the text
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)

    verb_tenses = []

    # Time indicators (adverbs, phrases)
    # future_time = {'tomorrow', 'soon', 'later', 'in the future'}
    # past_time = {'yesterday', 'last', 'ago', 'previously', 'earlier'}
    # present_time = {'now', 'currently', 'at the moment', 'as we speak'}

    # Check for time-related words
    text_lower = text.lower()
    # if any(word in text_lower for word in future_time):
    #     verb_tenses.append('Future')
    # if any(word in text_lower for word in past_time):
    #     verb_tenses.append('Past')
    # if any(word in text_lower for word in present_time):
    #     verb_tenses.append('Present')
    # if "was" in text_lower or "were" in text_lower:
    #     return "past"

    # Helper function to check for auxiliary/modal verbs
    def has_auxiliary(aux_list):
        return any(aux in text_lower for aux in aux_list)

    # Iterate through words with their POS tags
    for i, (word, tag) in enumerate(tagged):
        if tag.startswith("VB"):  # Checking for verb forms
            # Present Continuous: "is/are + VBG"
            if tag == "VBG" and i > 0 and tagged[i - 1][0].lower() in ["is", "are"]:
                verb_tenses.append("Present")  ####

            # Past Continuous: "was/were + VBG"
            elif tag == "VBG" and i > 0 and tagged[i - 1][0].lower() in ["was", "were"]:
                verb_tenses.append("Present")

            # Future Continuous: "will be + VBG"
            elif (
                tag == "VBG"
                and i > 1
                and tagged[i - 2][0].lower() == "will"
                and tagged[i - 1][0].lower() == "be"
            ):
                verb_tenses.append("Present")

            # "Going to" Future: "am/is/are going to + VB"
            elif (
                word.lower() == "going"
                and i < len(tagged) - 1
                and tagged[i + 1][0].lower() == "to"
            ):
                verb_tenses.append("Present")

            # Future Simple: "will + VB"
            elif i > 0 and tagged[i - 1][0].lower() == "will":
                verb_tenses.append("present")

            # Past Simple: "baked", "traveled" (VBD)
            elif tag == "VBD":
                verb_tenses.append("Past")

            # Present Simple: "walks", "runs", "eats" (VBP, VBZ)
            elif tag in ["VBP", "VBZ"]:
                verb_tenses.append("Present")

            # Past Participle: "was analyzed"
            elif tag == "VBN" and has_auxiliary(["was", "were"]):
                verb_tenses.append("Past")

            # Present Perfect: "has analyzed"
            elif tag == "VBN" and has_auxiliary(["has", "have"]):
                verb_tenses.append("Present")

            # Future Perfect: "will have analyzed"
            elif tag == "VBN" and has_auxiliary(["will have"]):
                verb_tenses.append("Present")

    # If no tenses were found, return "unknown"
    # if not verb_tenses:
    #     return "past"

    # Use Counter to determine the most common tense
    tense_counts = Counter(verb_tenses)
    try:
        primary_tense = tense_counts.most_common(1)[0][0]
    except IndexError:
        return "unknown"

    # Confidence calculation
    total_verbs = sum(tense_counts.values())
    # confidence = tense_counts.most_common(1)[0][1] / total_verbs
    if total_verbs == 0:
        return "unknown"
    # if total_verbs<10:
    #     print(primary_tense)
    #     print(text)

    # If confidence is too low, return "unknown"
    # if confidence < threshold:
    #     # print(primary_tense)
    #     # print(text)

    #     return "past"

    return primary_tense.lower()

In [26]:
analyze_sentence_tense("".join((examples[25]["content"])))

'past'

In [ ]:
# test_data = load_from_pickle("../data/test_dataset_2015.pkl")

# df_test_data = pd.DataFrame(test_data,index=["xml"],).T.reset_index()
# df_test_data.columns = ["patentnumber","xml"] 


# df = read_xlsb_file()

# df[df.issueyear == 2012].to_csv("freilichdataet_2012.csv")
df2 = pd.read_csv("freilichdataet_2011.csv")
df2["patentnumber"] = df2["patentnumber"].astype(str).transform(lambda x: x.replace(".0", ""))
df2["patentnumber"] = df2["patentnumber"].apply(remove_leadiong_zeros)
df2.head()

In [6]:
import sqlite3 as sql
import pandas as pd
import numpy as np
def test_algorithm(year):
    conn = sql.connect("../app/db/patents.db")
    cursor = conn.cursor()

    # Execute query and fetch results
    stats = cursor.execute("select * from patent_statistics").fetchall()
    stats_df = pd.DataFrame(stats, columns=["year", "patentnumber", "prophetic_patents", "nonprophetic_patents","unknown_patents"])
    conn.close()

    df_year = df2[df2.issueyear == year][["patentnumber","prophetic","nonprophetic","issueyear"]]
    df_year["patentnumber"] = df_year["patentnumber"].astype(str).transform(lambda x: x.replace(".0", ""))

    merged = stats_df.merge(df_year, on="patentnumber")
    print(len(merged))
    # final = dic_to_dic_w_tense_test(doc_w_exp,threshold=0)
    # df_final = pd.DataFrame(final).T.reset_index()
    df_final = merged
    #df_final.columns = ["patentnumber","past","present","Unknown"]
    df_final["patentnumber"] = df_final["patentnumber"].apply(remove_leadiong_zeros)
    df_check =merged
    #df_check["past"] = df_check["past"] + df_check["Unknown"]
    df_check["Total_Extracted"] = df_check["nonprophetic_patents"] + df_check["prophetic_patents"]  #+ df_check["Unknown"]
    df_check["Total_Freilich"] = df_check["prophetic"] + df_check["nonprophetic"]
    df_check["prophetic_error"] = np.sqrt((df_check["prophetic"] - df_check["prophetic_patents"])**2)
    df_check["nonprophetic_error"] = np.sqrt((df_check["nonprophetic"] - df_check["nonprophetic_patents"])**2)
    df_check["Total_Mean_error"] = np.sqrt((df_check["Total_Freilich"] - df_check["Total_Extracted"])**2)
    df_check["Sum_error"] = df_check["prophetic_error"] + df_check["nonprophetic_error"] + df_check["Total_Mean_error"]
    print(f"Number of exact matches: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])} out of {len(df_check)} , Percentage: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])/len(df_check)*100}")
    print(f"Number of exact num of patent extracted: {len(df_check[(df_check.Total_Extracted == df_check.Total_Freilich)])} out of {len(df_check)}")
    print(f"Avg Total error: {df_check['Total_Mean_error'].mean()}, num of corrects:  {df_check[df_check['Total_Mean_error'] == 0].shape[0]}")
    print(f"Avg Total prophetic error: {df_check['prophetic_error'].mean()}, num of corrects: {df_check[df_check['prophetic_error'] == 0].shape[0]}")
    print(f"Avg Total nonprophetic error: {df_check['nonprophetic_error'].mean()}, num of corrects: {df_check[df_check['nonprophetic_error'] == 0].shape[0]}")
    print(f"{df_check['Sum_error'].sum()}")
    print(f"Sum of number of Unknowns: {df_check['unknown_patents'].sum()}")
    print(f"Number of patets with experiments extracted {len(df_check)}")
    return df_check
df_check = test_algorithm(2011)
df_check.sort_values("Sum_error",ascending=False).head(20)
# 2011
# 943
# Number of exact matches: 628 out of 943 , Percentage: 66.59597030752916
# Number of exact num of patent extracted: 796 out of 943
# Avg Total error: 1.6246023329798516, num of corrects:  796
# Avg Total prophetic error: 1.1018027571580065, num of corrects: 721
# Avg Total nonprophetic error: 2.482502651113468, num of corrects: 631
# 4912.0
# Sum of number of Unknowns: 1065
# Number of patets with experiments extracted 943


# 2015
# 996
# Number of exact matches: 636 out of 996 , Percentage: 63.85542168674698
# Number of exact num of patent extracted: 828 out of 996
# Avg Total error: 0.7710843373493976, num of corrects:  828
# Avg Total prophetic error: 1.3744979919678715, num of corrects: 736
# Avg Total nonprophetic error: 2.0973895582329316, num of corrects: 642
# 4226.0
# Sum of number of Unknowns: 461
# Number of patets with experiments extracted 996

943
Number of exact matches: 628 out of 943 , Percentage: 66.59597030752916
Number of exact num of patent extracted: 796 out of 943
Avg Total error: 1.6246023329798516, num of corrects:  796
Avg Total prophetic error: 1.1018027571580065, num of corrects: 721
Avg Total nonprophetic error: 2.482502651113468, num of corrects: 631
4912.0
Sum of number of Unknowns: 1065
Number of patets with experiments extracted 943


,year,patentnumber,prophetic_patents,nonprophetic_patents,unknown_patents,prophetic,nonprophetic,issueyear,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error,Sum_error
268,1250,7879893,32,75,558,13.0,652.0,2011.0,107,665.0,19.0,577.0,558.0,1154.0
472,1507,7872139,16,445,147,0.0,608.0,2011.0,461,608.0,16.0,163.0,147.0,326.0
468,1503,7872126,67,117,0,218.0,0.0,2011.0,184,218.0,151.0,117.0,34.0,302.0
705,1858,7863292,44,168,41,0.0,253.0,2011.0,212,253.0,44.0,85.0,41.0,170.0
427,1462,7872010,18,180,31,0.0,255.0,2011.0,198,255.0,18.0,75.0,57.0,150.0
566,1656,7875639,3,373,72,0.0,448.0,2011.0,376,448.0,3.0,75.0,72.0,150.0
859,2094,7868037,39,72,1,104.0,8.0,2011.0,111,112.0,65.0,64.0,1.0,130.0
252,1234,7879849,5,184,0,0.0,245.0,2011.0,189,245.0,5.0,61.0,56.0,122.0
424,1459,7872004,0,11,53,0.0,64.0,2011.0,11,64.0,0.0,53.0,53.0,106.0
311,1294,7880012,42,40,0,0.0,82.0,2011.0,82,82.0,42.0,42.0,0.0,84.0


In [11]:
df_check[(df_check.Total_Extracted<df_check.Total_Freilich)]

,year,patentnumber,prophetic_patents,nonprophetic_patents,unknown_patents,prophetic,nonprophetic,issueyear,Total_Extracted,Total_Freilich,prophetic_error,nonprophetic_error,Total_Mean_error,Sum_error
0,383,RE042127,0,10,0,0.0,12.0,2011.0,10,12.0,0.0,2.0,2.0,4.0
9,400,7883615,0,2,0,0.0,3.0,2011.0,2,3.0,0.0,1.0,1.0,2.0
11,404,7883639,8,0,0,8.0,1.0,2011.0,8,9.0,0.0,1.0,1.0,2.0
16,415,7883686,0,4,0,0.0,6.0,2011.0,4,6.0,0.0,2.0,2.0,4.0
19,418,7883691,7,30,3,0.0,40.0,2011.0,37,40.0,7.0,10.0,3.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,2133,7868126,0,21,0,0.0,28.0,2011.0,21,28.0,0.0,7.0,7.0,14.0
896,2134,7868128,1,48,0,0.0,52.0,2011.0,49,52.0,1.0,4.0,3.0,8.0
932,2170,7868204,1,87,0,3.0,95.0,2011.0,88,98.0,2.0,8.0,10.0,20.0
935,2173,7868210,13,12,5,10.0,25.0,2011.0,25,35.0,3.0,13.0,10.0,26.0


In [3]:
test_dataset = read_json("patent_xmls_2010.json")
test_dataset_df = pd.DataFrame.from_dict(test_dataset, orient='index').reset_index()
test_dataset_df.columns = ['patentnumber', 'xml']
test_dataset_df.head()

,patentnumber,xml
0,7641702,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
1,7641704,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
2,7641709,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
3,7641721,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."
4,7641723,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<!DOCT..."


In [4]:
merged = df.merge(df2, on="patentnumber")
print(len(merged))
merged.head()

NameError: name 'df' is not defined

In [23]:
import os 
import re
from bs4 import BeautifulSoup

folder_path = "../app/data/patent_grants_2011"
file_names = os.listdir("../app/data/patent_grants_2011")
doc_w_exp = {}
for i, file in enumerate(file_names):
    all_xml_parts = []
    if file.endswith(".xml"):
        print(f"Processing {file}... ({i + 1}/{len(file_names)})")
        file_path = os.path.join(folder_path, file)
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
                parts = content.split('<?xml version="1.0" encoding="UTF-8"?>')
                parts = [p for p in parts if p.strip()]
                all_xml_parts.extend(parts)
        except Exception as e:
            print(f"Error processing {file}: {str(e)}")
        # xml_no_dup = remove_duplicate_docs(all_xml_parts)
        # print(f"Num of duplicates removed: {len(all_xml_parts) - len(xml_no_dup)} out of {len(all_xml_parts)}")
        for j, xml in enumerate(all_xml_parts):
            if j % 100 == 0 and j > 1:
                print(f"Processed {j}/{len(all_xml_parts)}...")
            if len(xml) <= 2000:
                pass
            s_tags = re.findall(r"<s\d+>.*?</s\d+>", xml)
            if len(s_tags) > 0 or '<sequence-cwu id="SEQLST-0">' in xml:
                pass

            heading = extract_experiments_w_heading(xml)

            # Process examples based on heading presence
            if heading and len(heading) == 1:
                #found_heading += 1
                examples = extract_examples_start_w_word(
                    heading[0].find_next_siblings()
                )
                if len(examples) == 0:
                    soup = BeautifulSoup(xml, "xml")
                    siblings = soup.find_all(["heading", "p"])
                    examples = extract_examples_start_w_word(siblings)
            else:
                #not_found_heading += 1
                soup = BeautifulSoup(xml, "xml")
                siblings = soup.find_all(["heading", "p"])
                examples = extract_examples_start_w_word(siblings)

            if len(examples) > 0:
                doc_num = remove_leadiong_zeros(find_doc_number(xml)[0])
                doc_w_exp[doc_num] = examples

Processing ipg110104.xml... (1/3)
Processed 100/5130...
Processed 200/5130...
Processed 300/5130...
Processed 400/5130...
Processed 500/5130...
Processed 600/5130...
Processed 700/5130...
Processed 800/5130...
Processed 900/5130...
Processed 1000/5130...
Processed 1100/5130...
Processed 1200/5130...
Processed 1300/5130...
Processed 1400/5130...
Processed 1500/5130...
Processed 1600/5130...
Processed 1700/5130...
Processed 1800/5130...
Processed 1900/5130...
Processed 2000/5130...
Processed 2100/5130...
Processed 2200/5130...
Processed 2300/5130...
Processed 2400/5130...
Processed 2500/5130...
Processed 2600/5130...
Processed 2700/5130...
Processed 2800/5130...
Processed 2900/5130...
Processed 3000/5130...
Processed 3100/5130...
Processed 3200/5130...
Processed 3300/5130...
Processed 3400/5130...
Processed 3500/5130...
Processed 3600/5130...
Processed 3700/5130...
Processed 3800/5130...
Processed 3900/5130...
Processed 4000/5130...
Processed 4100/5130...
Processed 4200/5130...
Processed

In [10]:
import numpy as np
# final = dic_to_dic_w_tense_test(doc_w_exp,threshold=0)
# df_final = pd.DataFrame(final).T.reset_index()
df_final = merged
#df_final.columns = ["patentnumber","past","present","Unknown"]
df_final["patentnumber"] = df_final["patentnumber"].apply(remove_leadiong_zeros)
df_check =merged
#df_check["past"] = df_check["past"] + df_check["Unknown"]
df_check["Total_Extracted"] = df_check["nonprophetic_patents"] + df_check["prophetic_patents"]  #+ df_check["Unknown"]
df_check["Total_Freilich"] = df_check["prophetic"] + df_check["nonprophetic"]
df_check["prophetic_error"] = np.sqrt((df_check["prophetic"] - df_check["prophetic_patents"])**2)
df_check["nonprophetic_error"] = np.sqrt((df_check["nonprophetic"] - df_check["nonprophetic_patents"])**2)
df_check["Total_Mean_error"] = np.sqrt((df_check["Total_Freilich"] - df_check["Total_Extracted"])**2)
df_check["Sum_error"] = df_check["prophetic_error"] + df_check["nonprophetic_error"] + df_check["Total_Mean_error"]
print(f"Number of exact matches: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])} out of {len(df_check)} , Percentage: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])/len(df_check)*100}")
print(f"Number of exact num of patent extracted: {len(df_check[(df_check.Total_Extracted == df_check.Total_Freilich)])} out of {len(df_check)}")
print(f"Avg Total error: {df_check['Total_Mean_error'].mean()}, num of corrects:  {df_check[df_check['Total_Mean_error'] == 0].shape[0]}")
print(f"Avg Total prophetic error: {df_check['prophetic_error'].mean()}, num of corrects: {df_check[df_check['prophetic_error'] == 0].shape[0]}")
print(f"Avg Total nonprophetic error: {df_check['nonprophetic_error'].mean()}, num of corrects: {df_check[df_check['nonprophetic_error'] == 0].shape[0]}")
print(f"{df_check['Sum_error'].sum()}")
print(f"Sum of number of Unknowns: {df._check['unknown_patents'].sum()}")
print(f"Number of patets with experiments extracted {len(df_check)}")
df_check.sort_values("Sum_error",ascending=False).head(20)

KeyError: 'nonprophetic_patents'

In [5]:
from utilities.nlp_processing import analyze_sentence_tense
def dic_to_dic_w_tense_test(doc_w_exp, threshold=0):
    dic = {}
    pattern = r"\(\d+\)\s*([A-Za-z0-9\-\(\)\{\},:;=\[\]\+\*\s\.\^\$\%]+(?:\.(?:sup|delta|Hz|NMR)[^\)]*)?)"

    for key, value in doc_w_exp.items():
        tense_counts = {"past": 0, "present": 0, "unknown": 0}

        if isinstance(value, list) and len(value) == 1:
            desc =  "".join(value[0]["content"]) # value[0]["title"] + "." +
            if len(desc) > threshold:
                tense = analyze_sentence_tense(desc)
                if tense != "unknown":
                    tense_counts[tense] += 1
                else:
                    matches = re.findall(pattern, desc)
                    if matches:
                        tense_counts["past"] += 1
                    else:
                        tense_counts["unknown"] += 1
                dic[key] = tense_counts

        elif isinstance(value, list) and len(value) > 1:
            for ls in value:
                desc = "".join(ls["content"]) #ls["title"] + "." + 
                if len(desc) > threshold:
                    if len(desc) > 0:
                        tense = analyze_sentence_tense(desc)

                        if tense != "unknown":
                            tense_counts[tense] += 1
                        else:
                            matches = re.findall(pattern, desc)
                            if matches:
                                tense_counts["past"] += 1
                            else:
                                tense_counts["unknown"] += 1
            dic[key] = tense_counts

        # elif isinstance(value, dict):
        #     print(value)
        #     for ex, desc in value.items():
        #         if len(desc) > threshold:
        #             tense = analyze_sentence_tense(desc)
        #             if tense != "unknown":
        #                 tense_counts[tense] += 1
        #             else:
        #                 matches = re.findall(pattern, desc)
        #                 if matches:
        #                     tense_counts["past"] += 1
        #                 else:
        #                     tense_counts["unknown"] += 1
        #     dic[key] = tense_counts
        # else:
        #     print(type(value))
        #     print(value)
        #     print(key)

    return dic

In [14]:
from bs4 import BeautifulSoup

def extract_examples_start_w_word(xml_siblings):
    examples = []
    current_example = None
    in_example = False

    for tag in xml_siblings:
        if tag.name == "heading":
            if (
                tag.text.strip().lower().startswith("example")
                or tag.text.strip().lower().startswith("experiment")
                or tag.text.strip().lower().startswith("test")
                or tag.text.strip().lower().startswith("trial")
                or "test" in tag.text.strip().lower()
                or "experiment" in tag.text.strip().lower()
                or "example" in tag.text.strip().lower()
                or "trial" in tag.text.strip().lower()
            ):
                in_example = True
                current_example = {
                    "number": tag.text.strip(),
                    "title": xml_siblings[xml_siblings.index(tag) + 1].text.strip(),
                    "content": [],
                }
                examples.append(current_example)
        elif tag.name == "heading" and (
            tag.text.strip().lower().startswith("example")
            or tag.text.strip().lower().startswith("experiment")
            or tag.text.strip().lower().startswith("test")
            or tag.text.strip().lower().startswith("trial")
            or "test" in tag.text.strip().lower()
            or "experiment" in tag.text.strip().lower()
            or "example" in tag.text.strip().lower()
            or "trial" in tag.text.strip().lower()
        ):
            in_example = False
        # else:
        #     # If we hit any other heading, stop collecting content
        #     in_example = False
        elif in_example and current_example is not None:
            current_example["content"].append(tag.text.strip())

    return examples
num_of_paterns = 5000
mae = 0
highest_difference = 0
found_heading = 0
not_found_heading = 0
gib = 0
short = 0
mostdifss = []
doc_w_exp = {}

for row in test_dataset_df.iterrows():
    xml = row[1]["xml"]
    if len(xml)>2000:
        s_tags = re.findall(r'<s\d+>.*?</s\d+>', xml)
        if len(s_tags) > 0:
            #print(f"Patent {row[1]['patentnumber']} is gibberish")
            gib+=1
        else:
            heading = extract_experiments_w_heading(xml)
            #janetsnumexamples = row[1]["prophetic"] + row[1]["nonprophetic"]

            if heading:
                found_heading += 1
                examples = extract_examples_start_w_word(heading[0].find_next_siblings())
                if len(examples)==0:
                    soup = BeautifulSoup(xml, 'xml')
                    siblings = soup.find_all(['heading', 'p'])
                    examples = extract_examples_start_w_word(siblings)
                numexamples = len(examples)
            else:
                not_found_heading += 1
                soup = BeautifulSoup(xml, 'xml')
                siblings = soup.find_all(['heading', 'p'])
                examples = extract_examples_start_w_word(siblings)
                numexamples = len(examples)



            # #difference = abs(numexamples-janetsnumexamples)
            # mae += difference
            # if difference > 0:
            #     mostdifss.append([difference, row[1]["patentnumber"]])
            # if difference > highest_difference:
            #     highest_difference = difference
            #     highest_difference_patent = row[1]["patentnumber"]
            if len(examples)>0:
                doc_w_exp[row[1]["patentnumber"]] = examples

    else:
        short+=1
        #print(f"skipping {row[1]['patentnumber']}, patent is too short")
        
        
print(f"found_heading: {found_heading}, not_found_heading: {not_found_heading}, gibberish: {gib},too_short: {short}")
print(f"{num_of_paterns} patents MAE: {mae/num_of_paterns}, total_error: {mae}, highest_difference: {highest_difference}, highest_difference_patent: ") #highest_difference_patent

found_heading: 7172, not_found_heading: 3713, gibberish: 3111,too_short: 379
5000 patents MAE: 0.0, total_error: 0, highest_difference: 0, highest_difference_patent: 


In [26]:
len(doc_w_exp)

1750

In [24]:
a = []
num_examples = 0
for k, pat_exs in doc_w_exp.items():
    num_examples += len(pat_exs)
    for ex in pat_exs:
        ola = "".join(ex["content"]) + ex["title"]
        if len(ola) == 0:
            #print(ex)
            continue
        a.append({
            'patent_number': k,
            'content_number': ex["number"]
        })
df_ex_types = pd.DataFrame(a)
val_counts= df_ex_types.content_number.value_counts()
valc_df = pd.DataFrame(val_counts).reset_index()

# Group by type and sum counts
def get_example_type(content_number):
    content_lower = content_number.lower()
    for prefix in start_w:
        if content_lower.startswith(prefix):
            return prefix
    for special in ls:
        if special in content_lower:
            return special
    return "other"

# Extract first word from each content_number
def get_first_word(text):
    return text.lower().split()[0]

# Get common prefixes (appearing more than 100 times)
common_prefixes = (df_ex_types['content_number']
                  .apply(get_first_word)
                  .value_counts()
                  .loc[lambda x: x > 100])

start_w = common_prefixes.index.tolist()
print("Discovered prefixes:")
print(common_prefixes/df_ex_types.shape[0] * 100)

# Function to detect patterns that occur in middle of text
def find_common_phrases(df, min_count=50):
    # Get all content numbers as lowercase
    texts = df['content_number'].str.lower()
    # Find phrases with 2 or more words that appear frequently
    phrases = texts[texts.str.contains(' .+ ')]  # Contains at least 2 spaces
    common_phrases = phrases.value_counts().loc[lambda x: x > min_count]
    return common_phrases.index.tolist()

ls = find_common_phrases(df_ex_types)
print("\nDiscovered special phrases:")
print(ls)

# Rest of your grouping code remains the same
valc_df['type'] = valc_df['content_number'].apply(get_example_type)
type_counts = valc_df.groupby('type')['count'].sum()

Discovered prefixes:
content_number
example        75.285318
comparative     4.797215
examples        3.159456
reference       2.611387
synthesis       2.134245
industrial      2.005287
preparation     1.966600
Name: count, dtype: float64

Discovered special phrases:
['comparative example 1', 'comparative example 2', 'comparative example 3', 'comparative example 4']


In [25]:
a = []
num_examples = 0
for k, pat_exs in doc_w_exp.items():
    num_examples += len(pat_exs)
    for ex in pat_exs:
        ola = "".join(ex["content"]) + ex["title"]
        if len(ola) == 0:
            continue
        a.append({
            'patent_number': k,
            'content_number': ex["number"]
        })

df_ex_types = pd.DataFrame(a)

# Function definitions remain the same
def get_example_type(content_number):
    content_lower = content_number.lower()
    for prefix in start_w:
        if content_lower.startswith(prefix):
            return prefix
    for special in ls:
        if special in content_lower:
            return special
    return "other"

def get_first_word(text):
    return text.lower().split()[0]

# Get common prefixes (appearing in more than 100 unique patents)
common_prefixes = (df_ex_types.groupby('patent_number')['content_number']
                  .apply(lambda x: x.apply(get_first_word).unique().tolist())
                  .explode()
                  .value_counts()
                  .loc[lambda x: x > 100])

start_w = common_prefixes.index.tolist()
print("Discovered prefixes (% of unique patents):")
print(common_prefixes/df_ex_types['patent_number'].nunique() * 100)

def find_common_phrases(df, min_count=50):
    # Get unique patent-phrase combinations
    patent_phrases = (df.groupby('patent_number')['content_number']
                     .apply(lambda x: x.str.lower().unique().tolist())
                     .explode())
    phrases = patent_phrases[patent_phrases.str.contains(' .+ ')]
    common_phrases = phrases.value_counts().loc[lambda x: x > min_count]
    return common_phrases.index.tolist()

ls = find_common_phrases(df_ex_types)
print("\nDiscovered special phrases:")
print(ls)

# Add type column to original dataframe
df_ex_types['type'] = df_ex_types['content_number'].apply(get_example_type)

# Count unique patents per type
type_counts = df_ex_types.groupby('type')['patent_number'].nunique().sort_values(ascending=False)
print("\nNumber of unique patents per type:")
print(type_counts)

Discovered prefixes (% of unique patents):
content_number
example        64.571429
industrial     17.771429
examples       16.914286
comparative    13.314286
Name: count, dtype: float64

Discovered special phrases:
['comparative example 1', 'comparative example 2', 'comparative example 3', 'comparative example 4']

Number of unique patents per type:
type
example                  1319
other                     476
industrial                311
comparative               233
comparative example 1       5
comparative example 2       3
Name: patent_number, dtype: int64


In [182]:
print("\nUnique content numbers:", len(df_ex_types['content_number'].unique()))
print("\nContent number frequencies:")
print(df_ex_types['content_number'].value_counts())

print("\nUnique patent numbers:", len(df_ex_types['patent_number'].unique()))
print("\nPatent number frequencies:")
print(df_ex_types['patent_number'].value_counts())


Unique content numbers: 23329

Content number frequencies:
content_number
Example 1                       6682
Example 2                       6024
Example 3                       5260
Example 4                       4561
Example 5                       3855
                                ... 
Example #N.1.1                     1
Example #O.1.1                     1
Example #X.1.1                     1
Example #38                        1
VI. Industrial Applicability       1
Name: count, Length: 23329, dtype: int64

Unique patent numbers: 7965

Patent number frequencies:
patent_number
8952157    1446
8987441    1040
8969587     996
8987242     901
9006265     800
           ... 
9034169       1
8956508       1
8956528       1
8956536       1
8986522       1
Name: count, Length: 7965, dtype: int64


INDUSTRIAL APPLICABILITY  :  30


In [13]:
print(f"Number of patents with difference > 0: {len(mostdifss)}")
mostdifss

Number of patents with difference > 0: 104


[[4.0, '8926548'],
 [1.0, '8926595'],
 [1.0, '8926732'],
 [1.0, '8926824'],
 [6.0, '8926862'],
 [4.0, '8926864'],
 [1.0, '8926872'],
 [1.0, '8926955'],
 [11.0, '8926966'],
 [7.0, '8926979'],
 [1.0, '8927015'],
 [1.0, '8927023'],
 [5.0, '8927036'],
 [2.0, '8927098'],
 [1.0, '8927150'],
 [2.0, '8927157'],
 [1.0, '8927236'],
 [4.0, '8927254'],
 [1.0, '8927429'],
 [1.0, '8927469'],
 [1.0, '8927479'],
 [1.0, '8927536'],
 [1.0, '8927541'],
 [1.0, '8927546'],
 [2.0, '8927568'],
 [4.0, '8927583'],
 [6.0, '8927588'],
 [3.0, '8927596'],
 [1.0, '8927617'],
 [1.0, '8927619'],
 [2.0, '8927631'],
 [1.0, '8927642'],
 [1.0, '8927647'],
 [1.0, '8927678'],
 [1.0, '8927681'],
 [2.0, '8927683'],
 [1.0, '8927698'],
 [7.0, '8927710'],
 [15.0, '8927721'],
 [1.0, '8927725'],
 [1.0, '8927730'],
 [1.0, '8927738'],
 [1.0, '8927746'],
 [1.0, '8927750'],
 [1.0, '8927777'],
 [1.0, '8927781'],
 [1.0, '8927801'],
 [9.0, '8927811'],
 [1.0, '8932406'],
 [3.0, '8932470'],
 [4.0, '8932480'],
 [2.0, '8932491'],
 [2.0, '89

In [15]:
# highest difference
df2[df2.patentnumber == "8933072"][["patentnumber","prophetic","nonprophetic"]]

,patentnumber,prophetic,nonprophetic
8285,8933072,0.0,9.0


In [ ]:
import numpy as np
final = dic_to_dic_w_tense_test(doc_w_exp,threshold=0)
df_final = pd.DataFrame(final).T.reset_index()
df_final.columns = ["patentnumber","past","present","Unknown"]
df_final["patentnumber"] = df_final["patentnumber"].apply(remove_leadiong_zeros)
df_check = df_final.merge(merged,on= "patentnumber",how="left")[["patentnumber","present","past","Unknown","prophetic","nonprophetic","allprophetic","someprophetic"]]
#df_check["past"] = df_check["past"] + df_check["Unknown"]
df_check["Total_Extracted"] = df_check["past"] + df_check["present"]  #+ df_check["Unknown"]
df_check["Total_Freilich"] = df_check["prophetic"] + df_check["nonprophetic"]
df_check["prophetic_error"] = np.sqrt((df_check["prophetic"] - df_check["present"])**2)
df_check["nonprophetic_error"] = np.sqrt((df_check["nonprophetic"] - df_check["past"])**2)
df_check["Total_Mean_error"] = np.sqrt((df_check["Total_Freilich"] - df_check["Total_Extracted"])**2)
df_check["Sum_error"] = df_check["prophetic_error"] + df_check["nonprophetic_error"] + df_check["Total_Mean_error"]
print(f"Number of exact matches: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])} out of {len(df_check)} , Percentage: {len(df_check[(df_check.prophetic_error ==0) & (df_check.nonprophetic_error ==0) & (df_check.Total_Mean_error ==0)])/len(df_check)*100}")
print(f"Number of exact num of patent extracted: {len(df_check[(df_check.Total_Extracted == df_check.Total_Freilich)])} out of {len(df_check)}")
print(f"Avg Total error: {df_check['Total_Mean_error'].mean()}, num of corrects:  {df_check[df_check['Total_Mean_error'] == 0].shape[0]}")
print(f"Avg Total prophetic error: {df_check['prophetic_error'].mean()}, num of corrects: {df_check[df_check['prophetic_error'] == 0].shape[0]}")
print(f"Avg Total nonprophetic error: {df_check['nonprophetic_error'].mean()}, num of corrects: {df_check[df_check['nonprophetic_error'] == 0].shape[0]}")
print(f"{df_check['Sum_error'].sum()}")
print(f"Sum of number of Unknowns: {df_check['Unknown'].sum()}")
print(f"Number of patets with experiments extracted {len(df_check)}")
df_check.sort_values("Sum_error",ascending=False).head(20)


# 1000
# Number of exact matches: 577 out of 799 , Percentage: 72.21526908635795
# Number of exact num of patent extracted: 694 out of 799
# Avg Total error: 0.5181476846057572, num of corrects:  694
# Avg Total prophetic error: 0.5857321652065082, num of corrects: 652
# Avg Total nonprophetic error: 0.9887359198998749, num of corrects: 587
# 1672.0
# Sum of number of Unknowns: 0
# Number of patets with experiments extracted 799

# 5000
# Number of exact matches: 2870 out of 3971 , Percentage: 72.27398640141023
# Number of exact num of patent extracted: 3461 out of 3971
# Avg Total error: 0.5560312263913372, num of corrects:  3461
# Avg Total prophetic error: 0.8645177537144296, num of corrects: 3209
# Avg Total nonprophetic error: 1.2976580206497104, num of corrects: 2942
# 10794.0
# Sum of number of Unknowns: 0
# Number of patets with experiments extracted 3971

SyntaxError: invalid syntax (2897131579.py, line 34)

In [ ]:
# patent 9102614 injanets says 4 examples but there are 161
# patent 9102599 in janets says 1 ,theres more than 1 actually 
# Some xml files are made with error : e.g. these dont have much information : skipping 9102601, 9102662,9102692
# these have gibberish: 9102724,9102705,9102727,9102628

In [ ]:
def process_siblings_test(xml_siblings):
    examples = []

    # Find all matching headings directly from xml_siblings
    example_headings = [
        tag
        for tag in xml_siblings
        if tag.name == "heading"
        and any(
            keyword in tag.text.strip().lower().replace(" ", "")
            for keyword in ["example", "experiment", "test"]
        )
        # and not any(
        #     excluded in tag.text.strip().lower().replace(" ", "")
        #     for excluded in ["reference", "preparation"]
        # )
    ]

    for heading in example_headings:
        current_content = []
        idx = xml_siblings.index(heading)

        # Get title from next heading if available
        title = ""
        if idx + 1 < len(xml_siblings) and xml_siblings[idx + 1].name == "heading":
            title = xml_siblings[idx + 1].text.strip()

        # Collect content until next example heading
        i = idx + 1
        while i < len(xml_siblings):
            if (
                xml_siblings[i].name == "heading"
                and any(
                    keyword in xml_siblings[i].text.strip().lower().replace(" ", "")
                    for keyword in ["example", "experiment", "test"]
                )
                # and not any(
                #     excluded in xml_siblings[i].text.strip().lower().replace(" ", "")
                #     for excluded in ["reference", "preparation"]
                # )
            ):
                break
            if xml_siblings[i].name == "p":
                current_content.append(xml_siblings[i].text.strip())
            i += 1

        examples.append(
            {"number": heading.text.strip(), "title": title, "content": current_content}
        )

    return examples if examples else None





In [6]:
def extract_examples_from_heading(heading):
    extracted_examples = []
    example_start_w = process_siblings(heading.find_next_siblings())
    if example_start_w:
        if not example_start_w[0]["content"]:
            extracted_ex_w_word = extract_examples_start_w_word(heading.find_next_siblings())
            if extracted_ex_w_word:
                if isinstance(extracted_ex_w_word, list):
                    if extracted_ex_w_word and len(extracted_ex_w_word[0]["content"])>0:
                        extracted_examples.append(extracted_ex_w_word)
                elif extracted_ex_w_word["content"]:
                    extracted_examples.append(extracted_ex_w_word)
        else:
            if len(example_start_w[0]["content"])>0:
                extracted_examples.append(example_start_w)
    else:
        extracted_ex_w_word = extract_examples_start_w_word(heading.find_next_siblings())
        if extracted_ex_w_word:
            if isinstance(extracted_ex_w_word, list):
                if extracted_ex_w_word and len(extracted_ex_w_word[0]["content"])>0:
                    extracted_examples.append(extracted_ex_w_word)
            elif extracted_ex_w_word["content"]:
                if len(extracted_ex_w_word["content"])>0:
                    extracted_examples.append(extracted_ex_w_word)
        else:
            num_dot_examples = extract_num_dot_examples(str(heading.find_next_siblings()))
            if num_dot_examples:
                extracted_examples.append(num_dot_examples)
    return extracted_examples

def extract_examples(xml):
    heading = extract_experiments_w_heading(xml)
    if heading:
        if len(heading) > 1:
            print(f"has more than 1 Examples heading, {i}")
        elif len(heading) == 1:
            return extract_examples_from_heading(heading[0])
        else:
            extracted_ex_w_word = extract_examples_w_word(xml)
            if extracted_ex_w_word:
                if isinstance(extracted_ex_w_word, list):
                    if extracted_ex_w_word and extracted_ex_w_word[0]["content"]:
                        return [extracted_ex_w_word]
                elif extracted_ex_w_word["content"]:
                    return [extracted_ex_w_word]
            else:
                example_start_w = process_siblings(heading[0].find_next_siblings())
                if example_start_w and example_start_w[0]["content"]:
                    return [example_start_w]
    else: 
        example_start_w = extract_examples_w_word(xml)
        if example_start_w:
            return [example_start_w]
        else:
            num_dot_examples = extract_num_dot_examples(xml)
            if num_dot_examples:
                return [num_dot_examples]
    return []

doc_w_exp = {}
for i, xml in enumerate(merged["xml"][:1000], start=1):
    if i % 1000 == 0:
        print(f"{i}/{len(merged.xml)} so far found {len(doc_w_exp)} docs with experiments")
    
    extracted_examples = extract_examples(xml)
    if extracted_examples:
        doc_w_exp[find_doc_number(xml)[0]] = extracted_examples

has more than 1 Examples heading, 349
has more than 1 Examples heading, 388
has more than 1 Examples heading, 689
has more than 1 Examples heading, 768
has more than 1 Examples heading, 781
has more than 1 Examples heading, 814
1000/25081 so far found 771 docs with experiments


In [73]:
def extract_examples_from_heading(heading):
    length_threshold = 30
    extracted_examples = []
    example_start_w = process_siblings(heading.find_next_siblings())
    if example_start_w:
        if not example_start_w[0]["content"]:
            extracted_ex_w_word = extract_examples_start_w_word(heading.find_next_siblings())
            if extracted_ex_w_word:
                if isinstance(extracted_ex_w_word, list):
                    extracted_ex_w_word = [ex for ex in extracted_ex_w_word if len(ex["content"])>length_threshold]
                    extracted_examples.append(extracted_ex_w_word)
                elif extracted_ex_w_word["content"]>length_threshold:
                    extracted_examples.append(extracted_ex_w_word)
        else:
            if isinstance(example_start_w, list):
                example_start_w = [ex for ex in example_start_w if len(ex["content"])>length_threshold]
                extracted_examples.append(example_start_w)
    else:
        extracted_ex_w_word = extract_examples_start_w_word(heading.find_next_siblings())
        if extracted_ex_w_word:
            if isinstance(extracted_ex_w_word, list):
                extracted_ex_w_word = [ex for ex in extracted_ex_w_word if len(ex["content"])>length_threshold]
                extracted_examples.append(extracted_ex_w_word)
            elif len(extracted_ex_w_word["content"])>length_threshold:
                extracted_examples.append(extracted_ex_w_word)
        else:
            num_dot_examples = extract_num_dot_examples(str(heading.find_next_siblings()))
            if isinstance(num_dot_examples, list):
                num_dot_examples = [ex for ex in num_dot_examples if len(ex["content"])>length_threshold]
                extracted_examples.append(num_dot_examples)
            elif isinstance(num_dot_examples, dict):
                num_dot_examples = [ex for ex in num_dot_examples.items() if len(ex["content"])>length_threshold]

                extracted_examples.append(num_dot_examples)
    return extracted_examples

def extract_examples(xml):
    length_threshold = 30
    heading = extract_experiments_w_heading(xml)
    if heading:
        if len(heading) > 1:
            print(f"has more than 1 Examples heading, {i}")
        elif len(heading) == 1:
            return extract_examples_from_heading(heading[0])
        else:
            pass
        # else:
        #     extracted_ex_w_word = extract_examples_w_word(xml)
        #     if extracted_ex_w_word:
        #         if isinstance(extracted_ex_w_word, list):
        #             extracted_ex_w_word = [ex for ex in extracted_ex_w_word if len(ex["content"])>length_threshold]
        #             return extracted_ex_w_word if extracted_ex_w_word
        #         elif extracted_ex_w_word["content"]>length_threshold:
        #             return extracted_ex_w_word if extracted_ex_w_word
        #     else:
        #         example_start_w = process_siblings(heading[0].find_next_siblings())
        #         if isinstance(example_start_w, list):
        #             example_start_w = [ex for ex in example_start_w if len(ex["content"])>length_threshold]
        #             return example_start_w if example_start_w
    else: 
        example_start_w = extract_examples_w_word(xml)
        if isinstance(example_start_w, list):
            example_start_w = [ex for ex in example_start_w if len(ex["content"])>length_threshold]
            return example_start_w
        else:
            num_dot_examples = extract_num_dot_examples(xml)
            if isinstance(num_dot_examples, list):
                num_dot_examples = [ex for ex in num_dot_examples if len(ex["content"])>length_threshold]
                return num_dot_examples

doc_w_exp = {}
for i, xml in enumerate(merged["xml"][:1000], start=1):
    if i % 100 == 0:
        print(f"{i}/{len(merged.xml)} so far found {len(doc_w_exp)} docs with experiments")
    
    extracted_examples = extract_examples(xml)
    if extracted_examples:
        doc_w_exp[find_doc_number(xml)[0]] = extracted_examples



100/25081 so far found 54 docs with experiments
200/25081 so far found 111 docs with experiments
300/25081 so far found 169 docs with experiments
has more than 1 Examples heading, 349
has more than 1 Examples heading, 388
400/25081 so far found 225 docs with experiments
500/25081 so far found 294 docs with experiments
600/25081 so far found 352 docs with experiments
has more than 1 Examples heading, 689
700/25081 so far found 395 docs with experiments
has more than 1 Examples heading, 768
has more than 1 Examples heading, 781
800/25081 so far found 463 docs with experiments
has more than 1 Examples heading, 814
900/25081 so far found 522 docs with experiments
1000/25081 so far found 575 docs with experiments


In [93]:

doc_w_exp = {}
for i, xml in enumerate(merged[:1999].xml.values, start=1):
    if i % 1000 == 0:
        print(f"{i}/{len(merged.xml)} so far found {len(doc_w_exp)} docs with experiments")
    
    extracted_examples = extract_examples(xml)
    if extracted_examples:
        doc_w_exp[find_doc_number(xml)[0]] = extracted_examples

has more than 1 Examples heading, 349
has more than 1 Examples heading, 388
has more than 1 Examples heading, 689
has more than 1 Examples heading, 768
has more than 1 Examples heading, 781
has more than 1 Examples heading, 814
1000/25081 so far found 769 docs with experiments
has more than 1 Examples heading, 1054
has more than 1 Examples heading, 1417
has more than 1 Examples heading, 1475
has more than 1 Examples heading, 1810


In [5]:
# save_as_pickle(doc_w_exp,"doc_w_exp_new_algorithm.pkl")
doc_w_exp = load_from_pickle("doc_w_exp_new_algorithm.pkl")

# save_as_json(doc_w_exp, "doc_w_exp_2015.json")
# doc_w_exp = read_json("../data/doc_w_exp_2015.json")

Loaded 18977 patents from doc_w_exp_new_algorithm.pkl


In [10]:
import re
import html
import unicodedata
from bs4 import BeautifulSoup

def clean_text_updated(text):
    """
    Clean text by removing HTML tags, special characters, extra spaces,
    and normalizing the content while keeping meaningful punctuation.

    Args:
        text (str): Input text to clean

    Returns:
        str: Cleaned text
    """
    if not isinstance(text, str):
        return ""

    # Decode HTML entities & remove HTML tags
    text = html.unescape(text)
    text = BeautifulSoup(text, "html.parser").get_text()

    # Normalize Unicode characters (e.g., é → e)
    text = unicodedata.normalize("NFKC", text)

    # Remove unwanted special characters but keep punctuation
    text = re.sub(r"[^\w\s.,!?'\-]", "", text)

    # Normalize spaces: remove multiple spaces, newlines, and tabs
    text = re.sub(r"\s+", " ", text).strip()

    return text

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

def check_tense_nltk_updated(sentence):
    words = word_tokenize(sentence)
    tagged = pos_tag(words)

    # Define tense categories
    past = {"VBD", "VBN"}
    present = {"VB", "VBG", "VBP", "VBZ"}
    future = {"MD"}

    tenses = {"past": 0, "present": 0, "future": 0}

    for i, (word, tag) in enumerate(tagged):
        # Count past tense words
        if tag in past:
            tenses["past"] += 1

        # Count present tense words
        elif tag in present:
            tenses["present"] += 1

        # Future tense handling
        elif tag in future:
            if word.lower() in {"will", "shall"}:
                # Ensure 'will' or 'shall' is followed by a verb (to confirm future tense)
                if i + 1 < len(tagged) and tagged[i + 1][1] in {"VB", "VBP"}:
                    tenses["future"] += 1

    return max(tenses, key=tenses.get) if max(tenses.values()) > 0 else "Unknown"



In [ ]:
import re
from utilities.nlp_processing import check_tense_nltk,clean_text

def tense_with_pattern(description):
    future_pattern = r'\b(?:will|would|shall|should)\b'
    past_pattern = r'\b(?:was|were|had|did)\b'

    future_matches = re.findall(future_pattern, description)
    past_matches = re.findall(past_pattern, description)
    if future_matches and past_matches:
        if len(future_matches) > len(past_matches):
            return "present"
        else:
            return "past"
    if future_pattern or past_pattern:
        if future_pattern:
            return "present"
        else:
            return "past"
    return "Unkown"


def dic_to_dic_w_tense(doc_w_exp,threshold = 50):
    dic = {}
    pattern = r'\(\d+\)\s*([A-Za-z0-9\-\(\)\{\},:;=\[\]\+\*\s\.\^\$\%]+(?:\.(?:sup|delta|Hz|NMR)[^\)]*)?)'
    pattern2 = r'\(\d+\)\s*(?:[A-Za-z]+\s*\d*\-?[A-Za-z]*[\(\{][^)]*[\)\}][^;]*|(?:\.\d*H\-?NMR[^;]*|\d+\.[a-zA-Z]*\d*))'


    for key, value in doc_w_exp.items():
        tense_counts = {"past": 0, "present": 0,"Unknown":0}
        
        if isinstance(value, list) and len(value) == 1 and len(value[0][0]["content"]) > threshold:
            desc = value[0][0]["title"] + "." + "".join(value[0][0]["content"])
            tense = check_tense_nltk_updated(clean_text(desc))
            if tense !=  "Unknown":
                tense_counts[tense] += 1
            else:
                matches = re.findall(pattern, desc)
                if matches:
                    tense_counts["past"] += 1
                else:
                    tense_counts["Unknown"] += 1
            dic[key] = tense_counts


        elif isinstance(value[0], list) and len(value[0]) > 1:
            for ls in value[0]:
                if len(ls["content"]) > threshold:
                    desc = "".join(ls["content"])
                    tense = check_tense_nltk_updated(clean_text(desc))
                    if tense !=  "Unknown":
                        tense_counts[tense] += 1
                    else:
                        matches = re.findall(pattern, desc)
                        if matches:
                            tense_counts["past"] += 1
                        else:
                            tense_counts["Unknown"] += 1
            dic[key] = tense_counts

        elif isinstance(value[0], dict):
            for ex, desc in value[0].items():
                if len(desc) > threshold:
                    tense = check_tense_nltk_updated(clean_text(desc))
                    if tense !=  "Unknown":
                        tense_counts[tense] += 1
                    else:
                        matches = re.findall(pattern, desc)
                        if matches:
                            tense_counts["past"] += 1
                        else:
                            tense_counts["Unknown"] += 1
            dic[key] = tense_counts
        else:
            print(type(value[0]))
            print(value[0])

    return dic

import re

def dic_to_dic_w_unknown_tense(doc_w_exp):
    dic = {}
    pattern = r'\(\d+\)\s*([A-Za-z0-9\-\(\)\{\},:;=\[\]\+\*\s\.\^\$\%]+(?:\.(?:sup|delta|Hz|NMR)[^\)]*)?)'
    pattern2 = r'\(\d+\)\s*(?:[A-Za-z]+\s*\d*\-?[A-Za-z]*[\(\{][^)]*[\)\}][^;]*|(?:\.\d*H\-?NMR[^;]*|\d+\.[a-zA-Z]*\d*))'

    for key, value in doc_w_exp.items():
        unknown_experiments = []  # Initialize a list to store the experiments classified as Unknown
        
        if isinstance(value[0], list) and len(value[0]) == 1:
            desc = value[0][0]["title"] + "." + "".join(value[0][0]["content"])
            tense = check_tense_nltk_updated(clean_text(desc))
            if tense == "Unknown":
                unknown_experiments.append(desc)

        elif isinstance(value[0], list) and len(value[0]) > 1:
            for ls in value[0]:
                desc = ls["title"] + "." + "".join(ls["content"])
                tense = check_tense_nltk_updated(clean_text(desc))
                if tense == "Unknown":
                    unknown_experiments.append(desc)

        elif isinstance(value[0], dict):
            for ex, description in value[0].items():
                tense = check_tense_nltk_updated(clean_text(description))
                if tense == "Unknown":
                    unknown_experiments.append(description)

        # Only add to the dictionary if there are "Unknown" experiments
        if unknown_experiments:
            dic[key] = unknown_experiments

    return dic


In [105]:
# 8946300 example 1-5
# 08946296 example 2 onwards 
# 08932484 examples 2 to 11


# 08933074:  
# Synthesis of 5,5-dimethyl-4-(pyridin-4-yl)-3-(4-(quinolin-2-ylmethoxy)phenyl) furan-2(5H)-one (Example 23)
# 5,5-dimethyl-4-(pyridin-4-yl)-3-(4-(quinolin-2-ylmethoxy)phenyl) furan-2(5H)-one (Example 23)

# 08945522:
# Example 2
# (69) 3-(3-(Triethoxysilyl)propylamino)phenyl benzoate and
# (140) Example 9
# (141) 4-(3-(Triethoxysilyl)propylamino)phenyl benzoate and
# (142) Example 10
# (143) 4-(Bis(3-(triethoxysilyl)propyl)amino)phenyl benzoate



SyntaxError: invalid syntax (1140748136.py, line 1)

In [ ]:
[8969641,8926966,8951515,8940436,9023999,9011956,9018396,9213027,9108964,9084992]#examples our algorithm didnt find but freilich did 
# many dont have an overarching examples section but have example 1, example 2 etc
# check these xml, the text has examples section:9194008,9018332,9054322,9150561,9213027, 

In [10]:
# import pprint as p
# p.pprint(merged[merged.patentnumber == "9018332"]["xml"].values[0])

# ERRORS IN FREILICH DATASET:
patentnumber 8957069: in freilich 108.0	nonproph and 0 prophetic but there are many more examples than 108, our algorithm: 267 nonprophetic and 1 prophetic and 3 unknown
patent number 8946443:we extracted 271	freilich: 108, this patent has refrtence examples, are we extracting these or not? 
patent number 8952010: same issue do we take refrence examples? 
patent 8933099: there are far more than 33 examples in this patent but freilich says 33, our algorithm found 131
what to do with Preparation examples?? e.g 8962612 in freilich has 6 we got 70.

# improvement for our algorithm
1. remove duplicates somehow, e.g in patent number 08987242, there some examples are extracted twice just because the name e.g example aa137 appears multiple times.
2. better classification of the patents. e.g patent 8987295 has 442, we extract the same number as frelich but we clasify differently.
3. 